In [29]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

from time import sleep
import tqdm
import json
import time
import re

import requests
from bs4 import BeautifulSoup
from html2text import html2text

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')

import numpy as np


In [30]:
category_links = [
    ('https://www.nn.ru/text/?rubric=economics&page=', 'economics'),
    ('https://www.nn.ru/text/?rubric=gorod&page=', 'gorod'),
    ('https://www.nn.ru/text/?rubric=culture&page=', 'culture'),
    ('https://www.nn.ru/text/?rubric=incidents&page=', 'incidents'),
    ('https://www.nn.ru/text/?rubric=politics&page=', 'politics'),
    ('https://www.nn.ru/text/?rubric=world&page=', 'world')
]

'''
sport - 300
economics - 935
gorod - много
science - 146
culture - много
incidents - много
politics - 1500
world - много
'''

'\nsport - 300\neconomics - 935\ngorod - много\nscience - 146\nculture - много\nincidents - много\npolitics - 1500\nworld - много\n'

In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


chrome_options = Options()
prefs = {"profile.managed_default_content_settings.images": 2,
            "profile.default_content_settings.cookies": 2,
            "profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument('--disable-application-cache')

driver = webdriver.Chrome(options=chrome_options)


def sleeepy():
    """Not to be banned by robot check"""
    sleep(np.random.randint(0, 50)/10)


category_res = {}
for link, category in category_links:
    news_links = []
    for page_id in range(1, 25):

        if page_id % 2 == 0: sleeepy()

        page_link = link + str(page_id)
        driver.get(page_link)
        page_news = driver.find_elements(By.CLASS_NAME, "h9Jmx")
        
        if len(page_news) == 0: break
        for element in page_news:
            news_links.append(element.find_element(By.TAG_NAME, 'a').get_attribute('href'))
        print(page_link, len(news_links))
    category_res[category] = news_links

    sleeepy()
driver.quit()

data = json.dumps(category_res)
with open("cache/links.json", "w") as f:
    f.write(data)

https://www.nn.ru/text/?rubric=economics&page=1 40
https://www.nn.ru/text/?rubric=economics&page=2 80
https://www.nn.ru/text/?rubric=economics&page=3 120
https://www.nn.ru/text/?rubric=economics&page=4 160
https://www.nn.ru/text/?rubric=economics&page=5 200
https://www.nn.ru/text/?rubric=economics&page=6 240
https://www.nn.ru/text/?rubric=economics&page=7 280
https://www.nn.ru/text/?rubric=economics&page=8 320
https://www.nn.ru/text/?rubric=economics&page=9 360
https://www.nn.ru/text/?rubric=economics&page=10 400
https://www.nn.ru/text/?rubric=economics&page=11 440
https://www.nn.ru/text/?rubric=economics&page=12 480
https://www.nn.ru/text/?rubric=economics&page=13 520
https://www.nn.ru/text/?rubric=economics&page=14 560
https://www.nn.ru/text/?rubric=economics&page=15 600
https://www.nn.ru/text/?rubric=economics&page=16 640
https://www.nn.ru/text/?rubric=economics&page=17 680
https://www.nn.ru/text/?rubric=economics&page=18 720
https://www.nn.ru/text/?rubric=economics&page=19 760
http

In [ ]:
with open("cache/links.json", "r") as f:
    links = json.load(f)

In [ ]:
leng = []
total = 0
for values in list(links.values()):
    len_values = len(values)
    leng.append(len_values)
    total += len_values
print(leng, total)

[40, 40, 40, 40, 40, 40] 240


In [ ]:
filename = "cache/news.json"

news = []
driver = webdriver.Chrome(chrome_options=chrome_options)
for category in links:
    print(f"================={category.upper()}=================")
    for link in tqdm.tqdm(links[category]):
        try:
            driver.get(link)
            page_text_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'qQq9J')]")
            raw_text = " ".join([i.get_property('innerText') for i in page_text_elements]).replace('\n', " ")
            text = re.sub(" +", " ", raw_text)
            key_words = driver.find_element(By.XPATH, "//meta[contains(@name, 'news_keywords')]").get_attribute('content')
            title = driver.find_element(By.XPATH, "//h1[contains(@itemprop, 'headline')]").get_property('innerText')
            news.append({
                    'article_id' : link,
                    'title': title,
                    'category': category,
                    'tags': key_words,
                    'text': text
                })
        except:
            continue
driver.quit()

with open(filename, "w", encoding='UTF-8') as f:
    f.write(json.dumps({
        'catalog': news
    }, ensure_ascii=False, indent=4))

=================ECONOMICS=================


100%|██████████| 1/1 [00:12<00:00, 12.22s/it]


=================GOROD=================


100%|██████████| 1/1 [00:05<00:00,  5.52s/it]


=================CULTURE=================


...

100%|██████████| 1/1 [00:03<00:00,  3.97s/it]


=================POLITICS=================


100%|██████████| 1/1 [00:00<00:00,  1.39it/s]


=================WORLD=================


100%|██████████| 1/1 [00:00<00:00,  1.80it/s]


In [ ]:
with open("cache/news.json", 'r', encoding="UTF-8") as f:
    news = pd.DataFrame(json.load(f).get('catalog'))

In [ ]:
news

,article_id,title,category,tags,text
0,https://www.nn.ru/text/economics/2023/01/17/71...,"Пенсионер показал, как выросли его коммунальны...",economics,"Квитанции,Пенсия,Платежки,Рост цен,Субсидии,ЖК...",Пока одни россияне молча отходят от шока после...
1,https://www.nn.ru/text/gorod/2023/01/17/71982941/,"«Возможно, в отместку». Нижегородским чиновник...",gorod,"Нижний Новгород,Кремль,Отопление,город","Завтра, 18 января, на время исчезнет отопление..."
2,https://www.nn.ru/text/culture/2023/01/17/7197...,Рядом с Меньшовым и Жванецким: как в Москве хо...,culture,культура,Сегодня в Москве прощаются с Инной Чуриковой. ...
3,https://www.nn.ru/text/incidents/2023/01/17/71...,В Верхних Печерах два огромных алабая напали н...,incidents,"Верхние Печеры,Нападение собак,Алабаи,происшес...",В Верхних Печерах два огромных алабая напали н...
4,https://www.nn.ru/text/politics/2023/01/17/719...,"«Один проливает кровь, а другой греет ляжки у ...",politics,"Нижний Новгород,Евгений Люлин,Политика,политика",Председатель нижегородского Заксобрания Евгени...
5,https://www.nn.ru/text/world/2023/01/18/71981936/,«Фиеста закончилась»: эмигрировавший блогер — ...,world,"Турция,Эмигранты,Эмиграция,ВНЖ,Частичная мобил...","Одной из стран, в которых решили обосноваться ..."
